# wav2vec-u CV-sv - GAN
> "GAN training for wav2vec-u on Common Voice Swedish"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [kaggle, colab, wav2vec-u]

The original attempt on [Kaggle](https://www.kaggle.com/jimregan/wav2vec-u-cv-swedish-gan) won't run because of an issue with CuDNN, but this notebook runs fine on Colab.

## Preparation

In [2]:
!pip install condacolab

In [3]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:36
🔁 Restarting kernel...


In [ ]:
%%capture
!conda install -c pykaldi pykaldi -y

In [5]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...
remote: Enumerating objects: 28296, done.
remote: Total 28296 (delta 0), reused 0 (delta 0), pack-reused 28296
Receiving objects: 100% (28296/28296), 11.77 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (21286/21286), done.


In [6]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 20.76 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [7]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [ ]:
%%capture
%cd /content/kenlm
!python setup.py install
%cd /tmp

In [2]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/content/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/content/fairseq'

In [3]:
%cd /content/fairseq/

/content/fairseq


In [ ]:
%%capture
!python setup.py install

In [5]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [6]:
%%capture
!pip install editdistance

https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

In [ ]:
%%capture
!pip install kaggle

In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [9]:
%cd /content

/content


In [10]:
!kaggle datasets download "jimregan/w2vu-cvsv-prepared-text"

 75% 13.0M/17.4M [00:00<00:00, 55.1MB/s]
100% 17.4M/17.4M [00:00<00:00, 64.5MB/s]


In [ ]:
%%capture
!unzip /content/w2vu-cvsv-prepared-text.zip

In [12]:
!kaggle datasets download -d jimregan/w2vu-cvsv-precompute-pca512-cls128-mean-pooled

 98% 386M/394M [00:04<00:00, 90.1MB/s]
100% 394M/394M [00:04<00:00, 102MB/s] 


In [13]:
%%capture
!unzip w2vu-cvsv-precompute-pca512-cls128-mean-pooled.zip

In [14]:
!rm *.zip

## GAN

In [15]:
import torch
torch.version.cuda

'10.1'

In [16]:
torch.backends.cudnn.version()

7603

In [17]:
%cd /content/fairseq

/content/fairseq


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
TASK_DATA=/content/precompute_pca512_cls128_mean_pooled
TEXT_DATA=/content/preppedtext/phones/
KENLM_PATH=/content/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX CUDA_LAUNCH_BLOCKING=1 fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.keep_last_epochs=5 \
	checkpoint.save_dir=/content/drive/MyDrive/w2vu \
	'common.seed=range(0,5)'

Writing rungan.sh


In [ ]:
!bash rungan.sh

[2021-06-04 00:06:14,189][fairseq.tasks.unpaired_audio_text][INFO] - REF: ɛ n f œ ʂ ə n a d ɵ ʂ ə k t f øː r d eː t s ɔ m h ɛ n d ə p oː ɕ œ r k ɔ n s ɛ t ə n
[2021-06-04 00:06:14,192][fairseq.tasks.unpaired_audio_text][INFO] - HYP: oː b iː ʃ œ m ɕ m œ ɕ ɪ ɵ ɕ ɵ m ɵ s ɵ uː ɵ s ɵ ɛ ʂ a tː sx
[2021-06-04 00:06:14,198][fairseq.tasks.unpaired_audio_text][INFO] - LM [REF]: -53.44462585449219, 0.05339602260269112
[2021-06-04 00:06:14,198][fairseq.tasks.unpaired_audio_text][INFO] - LM [HYP]: -61.104984283447266, 0.006571721232914821
[2021-06-04 00:06:14,844][valid][INFO] - {"epoch": 8, "valid_loss": "0.93", "valid_ntokens": "3039.79", "valid_nsentences": "144.214", "valid_lm_score_sum": "-71760.8", "valid_num_pred_chars": "28972", "valid_vocab_seen_pct": "0.949477", "valid_uer": "92.9812", "valid_weighted_lm_ppl": "229.386", "valid_lm_ppl": "206.793", "valid_wps": "15426", "valid_wpb": "3039.8", "valid_bsz": "144.2", "valid_num_updates": "128", "valid_best_weighted_lm_ppl": "189.002"}
[2021-0